## GreenBotanix.ВКРС

This is a draft version of a neural network to identify home plants by their picture.


Feedback: https://t.me/PVZaikin

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2, decode_predictions
import numpy as np
import matplotlib.pyplot as plt

Проверка предобученной модели:

In [16]:
# Загрузка предобученной модели MobileNetV2
model = MobileNetV2(weights='imagenet')

# Загрузка изображения и предобработка
img_path = "C:\\Users\\ahoy.captain\\Desktop\\GreenBot\\Pictures\\flower2.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Получение предсказаний
predictions = model.predict(img_array)
predicted_class = decode_predictions(predictions, top=1)[0][0]

# Вывод названия класса
print("Predicted plant: ", predicted_class[1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted plant:  daisy


Обучающие сеты:

In [17]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'C:\\Users\\ahoy.captain\\Desktop\\GreenBot\\Pictures\\TrainDATA',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'C:\\Users\\ahoy.captain\\Desktop\\GreenBot\\Pictures\\ValidationDATA',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


Дообучение:

In [18]:
# Загрузка предобученной модели MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Замораживаем веса базовой модели
base_model.trainable = False

# Число классов:
num_classes=10

# Добавление своего классификатора поверх базовой модели
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 12s 1us/step


ValueError: Must provide at least one structure

Проверка:

In [ ]:
# Оценка производительности модели
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy}')